# Notebook 1: Describing network data
#### Econometric Methods for Networks
#### _Singapore Management University (SMU) May 29th to June 1st, 2017_
##### _Bryan S. Graham, UC - Berkeley, bgraham@econ.berkeley.edu_
<br>
<br>
This is the first of a series of iPython Jupyter notebooks designed to accompany a series of instructional lectures given at the Singapore Management University (SMU) from May 29th to June 1st, 2017. The scripts below were written for Python 3.6. The Anaconda distribution of Python, available at https://www.continuum.io/downloads, comes bundled with most the scientific computing packages used in these notebooks.
<br>
<br>
For more information about the course please visit my webpage at http://bryangraham.github.io/econometrics/.
<br>
<br>


I begin by importing several key packages. The _numpy_ and _scipy_ libraries include a core set of scientific computing tools. The _pandas_ package is useful for data organization and analysis, while _matplotlib_ is Python's basic plotting add-on. Finally the _networkx_ package includes functionality for analysis, and also visualizing, network data.

In [1]:

# Division of two integers in Python 2.7 does not return a floating point result. The default is to round down 
# to the nearest integer. The following piece of code changes the default (redundant with 3.6 update).
from __future__ import division

# Direct Python to plot all figures inline (i.e., not in a separate window)
%matplotlib inline

# Main scientific computing modules
import numpy as np
import scipy as sp
import pandas as pd

# Import matplotlib 
import matplotlib.pyplot as plt

# networkx module for the analysis of network data
import networkx as nx


The following code snippet should be edited to point to wherever you have saved the instructional datasets for the course and/or want to store the figures generated in this notebook. The "data" string points to where the data files are saved. The "graphics" string points to where the figures generated below are saved.

In [2]:
# Directory where graphics files will be saved (and where example data are stored)
data =     '/Users/bgraham/Dropbox/Teaching/Short_Courses/St_Gallen/Data/'
graphics = '/Users/bgraham/Dropbox/Teaching/Short_Courses/SMU_CUHK/NotForRepo/'

To introduce some basic ideas of network analysis as well as associated Python functionality we will use the Nyakatoke risk sharing network data collected by Joachim de Weerdt. The de Weerdt (2004) paper in the course syllabus describes the dataset in some detail. There are four different files, we will mostly use the _Nyakatoke individual.dta_ and _Nyakatoke dyadic.dta_ files. The files are available for download from https://www.uantwerpen.be/en/staff/joachim-deweerdt/.

In [3]:
Nyakatoke_ind = pd.read_stata(data+"Nyakatoke/Nyakatoke individual.dta")
Nyakatoke_dyad = pd.read_stata(data+"Nyakatoke/Nyakatoke dyadic.dta")
Nyakatoke_hh = pd.read_stata(data+"Nyakatoke/Nyakatoke household.dta")
Nyakatoke_dir = pd.read_stata(data+"Nyakatoke/Nyakatoke directed.dta")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/bgraham/Dropbox/Teaching/Short_Courses/St_Gallen/Data/Nyakatoke/Nyakatoke individual.dta'

Here are the first few lines of the _Nyakatoke dyadic.dta_ file. Each row corresponds to a dyad -- that is a pair of households, whether connected or not.

In [ ]:
Nyakatoke_dyad.head()

In order to merge the individual and dyadic data we need to perform a few string comprehensions in order to link records. We can then aggregate the individual data up to the household level and perform a many-to-one merge of the dyadic with the household data (since the same household will belong to multiple dyads).

In [ ]:
Nyakatoke_ind['hh1']=Nyakatoke_ind['hhind'].apply(str).str[2:5]      # Create household ID number
Nyakatoke_ind['iid']=Nyakatoke_ind['hhind'].apply(str).str[5:7]      # Create individual ID number

# Find age of household head
Nyakatoke_ind['head_age'] = None
Nyakatoke_ind.loc[Nyakatoke_ind['iid']=='01', 'head_age'] = Nyakatoke_ind['age']

# Find sex of household head
gender_dict = {'male' : 1, 'female' : 0}
Nyakatoke_ind['head_sex'] = np.nan
Nyakatoke_ind.loc[Nyakatoke_ind['iid']=='01', 'head_sex'] = Nyakatoke_ind['sex'].map(gender_dict)

In [ ]:
# Create household-level dataset
grouped = Nyakatoke_ind[['head_age','head_sex','education']].groupby(Nyakatoke_ind['hh1'])
hh = grouped.max()
hh.reset_index(inplace=True)

In [ ]:
# Now work with the dyadic dataframe
# Convert hh1 to string and add leading zeros (to matched with hh dataframe created above)
Nyakatoke_dyad.loc[:,'hh1'] = Nyakatoke_dyad['hh1'].apply(str).str.zfill(5)

# remove decimal and digits to its right
Nyakatoke_dyad.loc[:,'hh1'] = Nyakatoke_dyad['hh1'].apply(str).str[0:3]

# Repeat conversions for hh2
Nyakatoke_dyad.loc[:,'hh2'] = Nyakatoke_dyad['hh2'].apply(str).str.zfill(5)
Nyakatoke_dyad.loc[:,'hh2'] = Nyakatoke_dyad['hh2'].apply(str).str[0:3]

In [ ]:
# Merge household head age, sex and education information into dyadic dataframe
# First for household 1 in the dyad
Nyakatoke_dyad = Nyakatoke_dyad.merge(hh, on = 'hh1', how = 'left', copy = False)
Nyakatoke_dyad.rename(columns={'head_age' : 'head_age1', 'head_sex' : 'head_sex1', 'education' : 'education1'}, inplace=True)

# Then for household 2 in the dyad
Nyakatoke_dyad = Nyakatoke_dyad.merge(hh, left_on='hh2', right_on = 'hh1', how = 'left', copy = False)

# NOTE: Last merge creates two instances of hh1 due to how the merge above was done. Drop the second instance and
#       rename the first
Nyakatoke_dyad.drop('hh1_y', axis=1, inplace=True)
Nyakatoke_dyad.rename(columns={'hh1_x' : 'hh1'}, inplace=True)

Nyakatoke_dyad.rename(columns={'head_age' : 'head_age2', 'head_sex' : 'head_sex2', 'education' : 'education2'}, inplace=True)

It is useful to have a household level measure of wealth. Here I use the conversion of land and livestock wealth into Tanzanian Shillings proposed by Comola and Fafchamps (2014 _Economic Journal_).

In [ ]:
# Comola and Fafchamps (2014) wealth formula
Nyakatoke_dyad['wealth1'] = (300000*Nyakatoke_dyad['land1'] + Nyakatoke_dyad['livestock1'])/100000
Nyakatoke_dyad['wealth2'] = (300000*Nyakatoke_dyad['land2'] + Nyakatoke_dyad['livestock2'])/100000

## Creating a graph object using _networkx_
The _networkx_ package provides functionality for defining, exploring and, to a lesser extent, visualizing network data. The next snippet of code defines a graph object using the Nyakatoke data. First I add each household in Nyakatoke to the graph object as a node, second I use *Nyakatoke\_dyad* dataframe to add edges or links where needed. Specifically, I use a Python list comprehension to quickly iterate through all dyads in the dataframe and add a link to the graph object as appropriate.

In [ ]:
# create networkx graph object
G=nx.Graph()

# add households to graph object
G.add_nodes_from(Nyakatoke_dyad['hh1'].unique())

# create edge list and add to graph 
G.add_edges_from([(row['hh1'], row['hh2']) for index, row in Nyakatoke_dyad.iterrows() if \
                  (row['links']!='no link') ])

The network visualization features of _networkx_ are basic, but sufficient for most purposes likely to arise in economics. In the picture generated below I color each node according to its wealth level. Berkeley Blue nodes correspond to "very poor" households; those with land and livestock wealth below 150,000 Tanzanian Shillings. Those colored California Gold are "poor" (wealth between 150,000 to 300,000 Shillings), Golden Gate colored nodes are "middle" wealth (300,000 to 600,000 Shillings), and those colored Lawrence are "wealthy" (wealth 600,000 Shillings or more).
<br>
<br>
The size of each node is proportional to is _degree_ (i.e., the number of links it has with other nodes).
<br>
<br>
The next few snippets of code add each households wealth category and degree as an attribute to our graph object. I use Python set operations to avoid repeating data since the same household appears in multiple dyads.

In [ ]:
# Group households by wealth
# Very Poor
set1 = set([row['hh1'] for index, row in Nyakatoke_dyad.iterrows() if (row['wealth1'] < 1.5)])
set2 = set([row['hh2'] for index, row in Nyakatoke_dyad.iterrows() if (row['wealth2'] < 1.5)])
very_poor = list(set1 | set2)

# Poor
set1 = set([row['hh1'] for index, row in Nyakatoke_dyad.iterrows() if (row['wealth1'] >= 1.5) & (row['wealth1'] < 3)])
set2 = set([row['hh2'] for index, row in Nyakatoke_dyad.iterrows() if (row['wealth2'] >= 1.5) & (row['wealth2'] < 3)])
poor = list(set1 | set2)

# Middle
set1 = set([row['hh1'] for index, row in Nyakatoke_dyad.iterrows() if (row['wealth1'] >= 3) & (row['wealth1'] < 6)])
set2 = set([row['hh2'] for index, row in Nyakatoke_dyad.iterrows() if (row['wealth2'] >= 3) & (row['wealth2'] < 6)])
middle = list(set1 | set2)

# Rich
set1 = set([row['hh1'] for index, row in Nyakatoke_dyad.iterrows() if (row['wealth1'] >= 6)])
set2 = set([row['hh2'] for index, row in Nyakatoke_dyad.iterrows() if (row['wealth2'] >= 6)])
rich = list(set1 | set2)

In [ ]:
# Categorize households into four bins based on total land and livestock wealth
# using the Comola and Fafchamps (2014, Economic Journal) scheme

# Construct dictionary with hh1 as 'key' and wealth as 'items'
hh_wealth = {} 
for index, row in Nyakatoke_dyad.iterrows():
    hh_wealth.setdefault(row['hh1'], []).append(row['wealth1'])
    hh_wealth.setdefault(row['hh2'], []).append(row['wealth2']) 

for hh in hh_wealth:
    
    # Remove duplicate wealth measures
    hh_wealth[hh] = list(set(hh_wealth[hh]))[0]
    
    # Catgorize into four groups
    if hh_wealth[hh] < 1.5:
        hh_wealth[hh] = 'very poor'
    elif (hh_wealth[hh] >= 1.5) & (hh_wealth[hh] < 3):
        hh_wealth[hh] = 'poor'
    elif (hh_wealth[hh] >= 3) & (hh_wealth[hh] < 6):
        hh_wealth[hh] = 'middle'
    elif hh_wealth[hh] >= 6:
        hh_wealth[hh] = 'rich'

# Attach wealth categorization to networkx graph        
nx.set_node_attributes(G, 'wealth', hh_wealth)        

# put together a color map, one color for a category
# Berkeley Blue: #003262
# California Gold: #FDB515
# Golden Gate: #ED4E33
# Lawrence: #00B0DA
color_map = {'very poor':'#003262', 'poor':'#FDB515', 'middle':'#ED4E33', 'rich':'#00B0DA'} 

# Construct degree attribute
nx.set_node_attributes(G, 'degree', nx.degree(G))

Now create the figure object and save it as a png file. Note this next code snippet requires _Graphviz_ and _PyGraphviz_ to be install. Graphviz is available for downloand at http://www.graphviz.org/. PyGraphiz may be installed using conda by typing "conda install -c pdrops pygraphviz=1.2" at the command line.

In [ ]:
# Plot Nyakatoke network
fig_nyakatoke = plt.figure(figsize=(6,4))

#----------------------------------------------------------#
#- Create custom legend                                   -# 
#----------------------------------------------------------#

import matplotlib.lines as mlines

# Legend items for node types
very_poor_patch = mlines.Line2D([], [], color='#003262', marker='o', linestyle='None',\
                                markersize=10, label= r'Wealth $<$ 150,000 TSh')
poor_patch      = mlines.Line2D([], [], color='#FDB515', marker='o', linestyle='None',\
                                markersize=10, \
                                label=r'150,000 TSh $\leqslant$ Wealth $<$ 300,000 TSh')
middle_patch    = mlines.Line2D([], [], color='#ED4E33', marker='o', linestyle='None',\
                                markersize=10, \
                                label=r'300,000 TSh $\leqslant$ Wealth $<$ 600,000 TSh')
rich_patch      = mlines.Line2D([], [], color='#00B0DA', marker='o', linestyle='None',\
                                markersize=10, label= r'Wealth $\geq$  600,000 TSh')

lgd = plt.legend(handles=[very_poor_patch, poor_patch, middle_patch, rich_patch], \
                          bbox_to_anchor=(0.5, -0.01), loc='upper center', fontsize=10, ncol=2, numpoints = 1) 

# Make sure plot does not have tick marks
plt.axis('off')           

# Use Graphviz to position nodes
# NOTE: need to call graphviz_layout with long name to avoid namespace error bug
node_pos = nx.drawing.nx_agraph.pygraphviz_layout(G,prog='sfdp')

# Plot network
ax = fig_nyakatoke.add_subplot(1,1,1)
from matplotlib import rc
rc('text', usetex=True)
plt.text(-30, 0, r'\underline{Note}: node sizes are proportional to household degree', fontsize=12)
nx.draw_networkx(G, pos=node_pos, with_labels=False, \
                 node_color = [color_map[G.node[node]['wealth']] for node in G], \
                 node_size = [G.node[node]['degree']*10 for node in G], \
                 edge_color='lightgrey', width=2)

# Set axis limits and suppress tick marks
#plt.ylim([0,200])
#plt.xlim([0,300])

ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)
plt.title('Nyakatoke Risk-Sharing Network', fontsize=14)

    
# Save figure
plt.tight_layout()
plt.savefig(graphics+'Nyakatoke_Network.png', bbox_extra_artists=(lgd,), bbox_inches='tight', dpi=500)

Frankly the network depiction is not especially informative. The figure does suggest that poorer households are located on the periphery of the network (i.e., have few links), whereas richer households have more links. It does make for a pretty picture however.
<br>
<br>
An important strand of network research takes the so-called _degree sequence_ of a network as its primary object of interest. This is motivated in part, by the fact that many other topological features of a network are fundamental constrained by its degree sequence (c.f, Albert and Barabasi, 2002 _Review of Modern Physics_, Graham, 2015 _Annual Review of Economics_; Wasserman and Faust, 1994, _Social Network Analysis_, Chapter 14).
<br>
<br>
The next snippet of code uses the nx.degree() function to calculate the degree sequence of the Nyakatoke network. And then plots it on a log-log scale.

In [ ]:
degree_sequence = pd.Series(sorted(nx.degree(G).values(),reverse=True)) # Degree sequence of network as Pandas series
N               = len(degree_sequence)                                  # Number of households in the network

# Complementary Cumulative Distribution Function for agent degrees
CCDF_degree = 1 - degree_sequence.value_counts().sort_index().cumsum()/N

print("Complementary Empirical CDF of Nyakatoke Degree Distribution")
print(CCDF_degree)

# Plot Nyakatoke Degree Distribution
fig_nyakatoke_degree = plt.figure(figsize=(6,4))
ax = fig_nyakatoke_degree.add_subplot(1,1,1)

plt.loglog(CCDF_degree,'-', marker='+', color='#00B0DA')
plt.title("Nyakatoke Degree Distribution")
plt.xlabel("Degree of household, d")
plt.ylabel(r"$\Pr (\mathrm{Degree_{i}} \geq d)$")

# Set axis limits and tick marks (log scale with tick labels in levels)
plt.ylim([0,1])
plt.xlim([0, 30])

ax.set_yticks([1e-2, 0.05, 0.1 ,0.25, 0.50, 0.75, 1])
ax.set_yticklabels([0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1])
ax.set_xticks([1, 5, 10, 15, 20, 25, 30])
ax.set_xticklabels([1, 5, 10, 15, 20, 25, 30])

# Annotate to show large number of low degree households coupled with a few hubs
plt.plot([5, 5], [0.01, CCDF_degree[5]], color='#ED4E33', linestyle=':', linewidth=1)
plt.plot([1, 5], [CCDF_degree[5], CCDF_degree[5]], color='#ED4E33', linestyle=':', linewidth=1)
plt.plot([20, 20], [0.01, CCDF_degree[20]], color='#ED4E33', linestyle=':', linewidth=1)
plt.plot([1, 20], [CCDF_degree[20], CCDF_degree[20]], color='#ED4E33', linestyle=':', linewidth=1)

# Save figure
plt.tight_layout()
plt.savefig(graphics+'Nyakatoke_Degree_Distribution.png', bbox_extra_artists=(lgd,), bbox_inches='tight', dpi=500)

If the degree sequence obeys a power law distribution then the log of the complementary CDF and the log of degree should lie on a straight line. There is not much evidence for power law behavior in the plot above, but it is consistent with a more qualitative feature of many social and economic networks. Specifically, there are many agents with only a small number of links along with a few "hub" agents with many links. In Nyakatoke over a quarter of households have five or fewer risk sharing links, while a little less than five percent of households have twenty or more links.
<br>
<br>
_Networkx_ also includes functionality for calculating basic summary statistics for a network. The next code snipped calculates the density, transitivity, average degree, standard deviation of degree and fraction of nodes in the giant component.

In [ ]:
print("Density          : " + "%3.4f" % nx.density(G))                                          # Density           
print("Transitivity     : " + "%3.4f" % nx.transitivity(G))                                     # Transitivity
print("Mean Degree      : " + "%3.4f" % np.mean(degree_sequence))                               # Mean Degree
print("Std. Dev. Degree : " + "%3.4f" % np.std(degree_sequence))                                # Std. of Degree
print("% in Giant Comp. : " + "%2.1f" % (100*len(max(nx.connected_components(G), key=len))/N))  # Giant component

The Nyakatoke network is relatively sparse. Less that 7 percent of all possible edges are present. Transitivity is about three times density. So there are many more closed triangles in the network than one would expect to see in a vanilla random graph with the same density.
<br>
<br>
The _eccentricity()_ function calculates the maximum path length from an agent in the network to all other agents in the network. If the graph is not connected this (maximal) distance will be infinite. Using the counter dictionary subclass we can quickly compute the frequency of different maximal "degrees of separation" in the Nyakatoke network.
<br>
<br>
The maximum path length from one agent to another in the Nyakatoke network is 5. This equals the _diameter_ of the network.

In [ ]:
ds = nx.eccentricity(G)

from collections import Counter

for distance, count in Counter(ds.values()).most_common():
    print("Number of agent at most " + str(distance) + " degrees from all other agents is " + str(count))


We can also count the frequency of different shortest path lengths in the network. For each pair agents there is a minimal distance between them. If they are directly connected this distance is one. If they share a friend in common is is two and so on.
<br>
<br>
The *shortest\_path\_length()* function returns a dictionary of dictionaries. There is one dictionary for each household. Each dictionary has households $j=1,..,N$ as keys and shortest path lengths to them as items. The follow snippet of code iterates over these objects and computes the frequency of different shortest path lengths between dyads.

In [ ]:
pl = nx.shortest_path_length(G)   # Returns a dictionary of dictionaries
                                  # Each dictionary gives the path length from household i to all households j=1,..,N 

shortest_path_length_counts = {}

for i, path_lengths_for_i in pl.items():
    for j, path_length in path_lengths_for_i.items():
        # If path_length not in dictionary add it as a key and set is item to zero (setdefault method)
        shortest_path_length_counts.setdefault(path_length, 0)
        shortest_path_length_counts[path_length] += 1

for path_length, count in shortest_path_length_counts.items():
    if path_length != 0:
        shortest_path_length_counts[path_length] = count // 2       # Divide by two to avoid double counting
    print(str(shortest_path_length_counts[path_length]) + " pairs of households are " + str(path_length) + " degrees apart")    

The Nyakatoke network shares another feature common to many real world social and economic networks. The network is relatively sparse, but simultaneously has small diameter. This goes back to Stanley Milgram's work in the 1960s. Only 490 our of 7021 possible links are present in the network. Density is about 7%. At the same time the shortest path length between any two pairs of households is quite small. While there are only 490 direct ties in the networks, many households share a link/friend in common (2666). The vast majority of dyads are at most three degrees apart.
<br>
<br>
Another commonly reported graph summary is the so called triad census. In the case of directed graphs there are 16 possible wirings of a three agent subgraph. See Wasserman and Faust (1994, Chapters 6 & 14) for background, theorizing and terminology. When the graph is undirected, there are only four possible triad configurations. The following snippet of code creates a figure illustrating each of the four wirings. 

In [ ]:
empty=nx.Graph()                        # Empty triad
empty.add_nodes_from([1,2,3])

one_edge=nx.Graph()                     # One edge triad       
one_edge.add_nodes_from(empty)
one_edge.add_edges_from([(1,2)])

two_star=nx.Graph()                     # Two star triad
two_star.add_nodes_from(empty)
two_star.add_edges_from([(1,2),(1,3)])

triangle=nx.Graph()                     # Triangle triad   
triangle.add_nodes_from(empty)
triangle.add_edges_from([(1,2),(1,3),(2,3)])

# Position nodes in the shape of a triangle
node_pos = {1: [1,1], 2: [2,0], 3: [0,0]} 

triad_census_fig = plt.figure(figsize=(6, 1.5))

# Empty triad
ax = triad_census_fig.add_subplot(1,4,1)
nx.draw_networkx(empty, pos=node_pos, with_labels=True, node_color='#FDB515', edge_color='#003262', width=3)
ax.axes.set_xlim([-0.5,2.5])
ax.axes.set_ylim([-0.5,1.5])
ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)
plt.title('Empty')

# One edge triad
ax = triad_census_fig.add_subplot(1,4,2)
nx.draw_networkx(one_edge, pos=node_pos, with_labels=True, node_color='#FDB515', edge_color='#003262', width=3)
ax.axes.set_xlim([-0.5,2.5])
ax.axes.set_ylim([-0.5,1.5])
ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)
plt.title('One edge')

# Two star triad
ax = triad_census_fig.add_subplot(1,4,3)
nx.draw_networkx(two_star, pos=node_pos, with_labels=True, node_color='#FDB515', edge_color='#003262', width=3)
ax.axes.set_xlim([-0.5,2.5])
ax.axes.set_ylim([-0.5,1.5])
ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)
plt.title('Two star')

# Triangle triad
ax = triad_census_fig.add_subplot(1,4,4)
nx.draw_networkx(triangle, pos=node_pos, with_labels=True, node_color='#FDB515', edge_color='#003262', width=3)
ax.axes.set_xlim([-0.5,2.5])
ax.axes.set_ylim([-0.5,1.5])
ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)
plt.title('Triangle')

# Save result
plt.tight_layout()
plt.savefig(graphics+'Triad_Census.png', dpi=500)  

Somewhat irritatingly _networkx_ only computes a triad census for directed graphs. However we can shoehorn a census for undirected graphs by using to_directed() method). This is not especially elegant, but does the trick.

In [ ]:
print("%s %s" % ('TRIAD'.ljust(10), 'COUNT'.rjust(15)))
print("--------------------------")
for triad, count in nx.triadic_census(G.to_directed()).items():
    if triad in ['003', '102', '201', '300']:
     print("%s %s" % (triad.ljust(10), '{:,d}'.format(count).rjust(15)))

There are 221,189 "empties" in the Nyakatoke network. A total of 48,245 triads with just one edge; 4,070 so called "two stars" and just 315 triangles. However transitivity is about three times density. The graph is clustered.
<br>
<br>
Recently, Matt Jackson and co-authors introduced the notion of links being "supported" (see Jackson, Rodriguez-Barrquer and Tan, 2012, _American Economic Review_. The notion of support is closely related to transitivity, but measured differently. We can calculate support using a few manipulations of the adjacency matrix.

In [ ]:
D = nx.adjacency_matrix(G) # Get Nyakatoke adjacency matrix as a Scipy sparse matrix object
D2 = D*D                   # Matrix square of D

# The {i,j} element of D2 gives the number of links agents i and j share in common
# Get multi-indices for lower triangle of N x N matrix
ij_LowTri = np.tril_indices(N, -1)

# Compute fraction of links that are supported
support =  np.sum((D.multiply(D2 != 0))[ij_LowTri]) / np.sum(D[ij_LowTri])

# Compute "average support" (see below)
avg_support =  np.sum((D.multiply(D2))[ij_LowTri]) / np.sum(D[ij_LowTri])

print("%3.1f" % (100*support) + " of all links in Nyakatoke are supported by a common friend.")
print("On average, linked households in Nyakatoke share " + "%3.1f" % avg_support + " friends in common.")

Links in Nyaktoke are heavily supported. Interestingly the figure of 80 percent is quite close to that across 75 villages in Karnataka in southern India as reported by Jackson, Rodriguez-Barrquer and Tan (2012). On average linked households in Nyakatoke share 1.9 friends in common.

### Save dyadic data and adjacency matrix for future use
We will revisit the Nyakatoke network in subsequent lectures. This final code snippet saves a dyadic dataset of links and attributes for the subset of households that have no missing data on key variables. It also saves a copy of the adjacency matrix.

In [ ]:
es = Nyakatoke_dyad[["hh1", "hh2", "links", "kinship", "distance", "clan1", "clan2", "wealth1", "wealth2", \
                     "religion1", "religion2", "education1", "education2", "head_age1", "head_age2", \
                     "head_sex1", "head_sex2"]] 

# Only retain complete cases
es = es.dropna().sort_values(['hh1', 'hh2'], ascending=[True, True])

# Save dataframe to cvs file
es.to_csv(data+"Created/Nyakatoke_Estimation_Sample.csv")

# Save adjacency matrix
from scipy import io
io.mmwrite(data+"Created/Nyakatoke_Adjacency.mtx", D)